# **DATA3888 Project: Optiver**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import polars as pl
from glob import glob
from sklearn.decomposition import PCA
from sklearn.preprocessing  import StandardScaler

In [2]:
csv_files = sorted(glob("Data/individual_book_train/*.csv"))

ldf = pl.scan_csv(
    csv_files,
    schema_overrides={         
        'time_id': pl.Int64,
        'seconds_in_bucket': pl.Int64,
        'bid_price1': pl.Float64,
        'ask_price1': pl.Float64,
        'bid_price2': pl.Float64,
        'ask_price2': pl.Float64,
        'bid_size1': pl.Int64,
        'ask_size1': pl.Int64,
        'bid_size2': pl.Int64,
        'ask_size2': pl.Int64,
        'stock_id': pl.Int64,
    },
    infer_schema_length=0
)

# df = ldf.collect()  
# df.write_parquet("Data/Combined_book_train.parquet", compression="snappy")

In [2]:
df = pl.read_parquet(
    "Data/Combined_book_train.parquet",
    use_pyarrow=True  
)

In [3]:
df = df.to_pandas()

In [16]:
window_size = 300   
df['window_start'] = (df['seconds_in_bucket'] // window_size) * window_size

agg = (
    df
    .groupby(['stock_id', 'time_id', 'window_start'], as_index=False)
    .agg({
        'bid_price1':  'mean',
        'ask_price1':  'mean',
        'bid_price2':  'mean',
        'ask_price2':  'mean',
        'bid_size1':   'sum',
        'ask_size1':   'sum',
        'bid_size2':   'sum',
        'ask_size2':   'sum',
    })
)

agg['seconds_in_bucket'] = agg['window_start']

In [17]:
agg["stock_id"].unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  26,  27,  28,
        29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
        42,  43,  44,  46,  47,  48,  50,  51,  52,  53,  55,  56,  58,
        59,  60,  61,  62,  63,  64,  66,  67,  68,  69,  70,  72,  73,
        74,  75,  76,  77,  78,  80,  81,  82,  83,  84,  85,  86,  87,
        88,  89,  90,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102,
       103, 104, 105, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       118, 119, 120, 122, 123, 124, 125, 126])

In [18]:
summary = (agg
    .groupby('stock_id')
    .agg(
        avg_bid_size1   = ('bid_size1', 'mean'),
        avg_ask_size1   = ('ask_size1', 'mean'),
        avg_bid_size2   = ('bid_size2', 'mean'),
        avg_ask_size2   = ('ask_size2', 'mean'),
        avg_bid_price1  = ('bid_price1','mean'),
        avg_ask_price1  = ('ask_price1','mean'),
    )
)

summary['spread']    = summary['avg_ask_price1'] - summary['avg_bid_price1']
summary['depth']     = summary[['avg_bid_size1','avg_ask_size1','avg_bid_size2','avg_ask_size2']].mean(axis=1)
summary['liq_score'] = summary['depth'] / summary['spread']

k = 30
top_liquid = summary.nlargest(k, 'liq_score').index.tolist()
print(top_liquid)

[32, 31, 77, 43, 29, 41, 108, 111, 47, 124, 46, 35, 119, 44, 125, 69, 21, 13, 86, 99, 63, 123, 14, 10, 120, 85, 105, 89, 74, 50]


In [19]:
df = agg.copy()

df['mid'] = (df['bid_price1'] + df['ask_price1']) / 2
df = df.sort_values(['stock_id','time_id','seconds_in_bucket'])
df['mid_prev'] = df.groupby('stock_id')['mid'].shift(1)
df['ret']     = np.log(df['mid'] / df['mid_prev'])

vol = df.groupby('stock_id')['ret'].var().rename('ret_var')

threshold = vol.quantile(0.75)
high_vol = vol[vol > threshold].index.tolist()
print(high_vol)

[1, 3, 6, 8, 9, 11, 16, 18, 27, 31, 33, 44, 56, 62, 63, 74, 75, 80, 81, 83, 88, 89, 90, 94, 97, 100, 112, 124]


In [20]:
summary = summary.join(vol)

X = StandardScaler().fit_transform(summary)
pca = PCA(n_components=2).fit(X)
scores = pca.transform(X)   

pc1_scores = pd.Series(np.abs(scores[:,0]), index=summary.index)
top_pca = pc1_scores.nlargest(30).index.tolist()
print(top_pca)

[31, 32, 77, 108, 41, 43, 29, 111, 47, 35, 37, 33, 98, 5, 103, 112, 18, 75, 4, 27, 30, 0, 88, 16, 62, 116, 113, 110, 126, 90]
